In [1]:
import os
import json
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct" #"nlpai-lab/KULLM3" #"rtzr/ko-gemma-2-9b-it"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True 
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSelfAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear(in_features=4096, out_features=14336, bias=False)
          (c_fc_1): Linear(in_features=4096, out_features=14336, bias=False)
          (c_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act): SiLU

In [3]:
with open("../data/filtered_exaone_restored.csv", "r") as f:
    df = pd.read_csv(f)

In [4]:
def inference_topic(text: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 같은 target 라벨을 가지고 있는 'ID' 'text' 'target'으로 이루어진 텍스트들 여러 개가 들어올 겁니다. 해당 입력의 target의 공통 주제 단 한가지를 들어온 텍스트들의 분석을 통해 유추하시오. 유추할 때는 모든 텍스트들을 고려하여서 가장 포괄적인 키워드를 뽑아야합니다. 키워드는 뉴스 카테고리 중 하나입니다. 설명을 붙이지 말고 유추한 키워드 단 한 개만을 출력하세요."},
    {"role": "user", "content": text}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    if '\n' in result:
        result = result.split("\n")[0]
    print(result)
    del inputs, outputs, generated_text
        
    return result

In [5]:
def filter(text: str, subject: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 같은 target 라벨을 가지고 있는 'ID' 'text' 'target'으로 이루어진 텍스트들 여러 개가 들어오고 그들의 공통 주제가 주어집니다. 들어온 텍스트들 중에는 공통 주제에 맞지 않는 텍스트들이 존재하는데 이들을 필터링하는 임무를 부여받았습니다. 당신은 주어진 텍스트들에서 이들을 정제한 후 그 결과를 설명을 붙이지 말고 'ID','text','target' 형태로 출력하세요."},
    {"role": "user", "content": text},
    {"role": "user", "content": subject}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=4096,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    del inputs, outputs, generated_text
        
    return result

In [9]:
def generate(subject: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 뉴스 카테고리 주제가 하나 들어올 떼 그 주제에 맞는 적절한 기사 제목을 설명을 붙이지 말고 단 한 개만 생성해줘. 신문 기사를 생성할 때 다양성을 많이 주었으면 좋겠어. 그리고 설명은 절대 붙이지 말고 신문 기사 제목만을 출력해줘."},
    {"role": "user", "content": subject}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    del inputs, outputs, generated_text
        
    return result

In [7]:
from tqdm.notebook import tqdm

filtered_datas = []
topics_dict = []
for i in tqdm(range(0,7), desc="label"):
    rows = df[df['target'] == i].sample(frac=1).reset_index(drop=True)
    keywords = {}
    context = ""
    for k, row in tqdm(rows.iterrows(), desc="Filter mis-labels", total=len(rows)):
        context += f"{row['ID']},{row['text']},{row['target']}\n"
        if (k + 1) % 60 == 0 or k == len(rows) - 1:
            topic = inference_topic(context)
            sub_topic = topic.split(" ")
            for sub in sub_topic:
                keywords[sub] = keywords.get(sub, 0) + 1
            context = ""
    keywords = dict(sorted(keywords.items(), key=lambda item: item[1], reverse=True))
    topics_dict.append(keywords)
    # for key in keywords.keys():
    #     if key not in topics:
    #         topics.append(key)
    #         break

topics = []
for i, sub1 in enumerate(topics_dict):
    nan = []
    candidates = []
    can_val = []
    for k, sub2 in enumerate(topics_dict):
        if i == k: continue
        skip_outer = False
        for key1 in sub1.keys():
            if key1 in topics: continue
            if key1 not in sub2.keys():
                if key1 not in candidates:
                    candidates.append(key1)
                    can_val.append(sub1[key1])
                break
            if key1 in sub2.keys():
                if sub1[key1] < sub2[key1]: 
                    if k > i: nan.append(key1)
                    continue
                if key1 not in candidates:
                    candidates.append(key1)
                    can_val.append(sub1[key1])

    mx = -1
    fnl = None
    for v, can in enumerate(candidates):
        if can in nan: continue
        if mx < v: 
            mx = v
            fnl = can
    topics.append(fnl)


print(topics)

    


label:   0%|          | 0/7 [00:00<?, ?it/s]

Filter mis-labels:   0%|          | 0/233 [00:00<?, ?it/s]

문화
경제
문화
경제 성장


Filter mis-labels:   0%|          | 0/228 [00:00<?, ?it/s]

스포츠
스포츠
스포츠
스포츠


...

스마트폰
기술 혁신
기술 혁신
전자 제품


Filter mis-labels:   0%|          | 0/230 [00:00<?, ?it/s]

경제
경제
경제
경제


Filter mis-labels:   0%|          | 0/222 [00:00<?, ?it/s]

정치
정치
정치
국제 정치
['문화', '스포츠', '정치', '사회', '기술', '경제', '국제']


In [10]:
generated_article = []
for idx, sub in tqdm(enumerate(topics), desc='gen_article', total=len(topics)):
    for k in tqdm(range(0, 200)):
        article = generate(sub)
        print(article)
        generated_article.append(
            {
                'ID': f"generated_by_exaone_{idx}_{k}",
                'text': article,
                'target': idx
            }
        )


gen_article:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

"전통과 현대의 조화: 서울 북촌 한옥마을의 새로운 예술 공간들"
"전통 궁중 음식, 현대 속으로: 왕실의 맛을 집에서"
"전통 춤의 현대적 재해석: 서울 페스티벌에서 만난 새로운 한류"
"전통 시장의 재발견: 지역 문화와 경제의 새로운 희망"
"한국 전통 공예, 세계 무대서 다시 빛나다"
"전통시장의 재발견: 숨겨진 예술과 맛의 향연"
"한류 열풍, K-팝에서 전통 예술까지: 한국 문화의 세계화 전략"
"전통 시장의 현대화: 지역 경제를 살리는 혁신의 물결"
"한국 전통 예술의 현대적 재탄생: 젊은 예술가들이 그리는 새로운 풍경"
"전통 음악 페스티벌, 서울서 개최...지역 예술가들의 숨은 보석 공개"
"한국 전통음악, 현대 무대서 부활: 전통과 혁신의 만남"
"한국 전통 춤, 현대 패션과 만나다: K-패션 속 전통의 재해석"
"한국 전통 예술의 현대적 변용: 젊은 예술가들의 혁신적 도전"
"전통 예술의 현대적 재해석: 지역 예술가들의 창의적 융합"
"전통과 현대의 만남: 도시 속에서 부활하는 한국 전통 예술"
"한국 전통 춤, 세계 무대에서 빛나다: UNESCO 인류무형문화유산 등재"
"전통 공예의 부활: 현대 디자이너들, 한국 전통 기술 되살려"
"한국 전통 예술의 현대적 재해석: 젊은 예술가들의 도전과 혁신"
"한류 열풍, K-팝의 글로벌 성공 비결 분석"
"한국 전통 음악, 현대 무대에 부활: 젊은 예술가들의 새로운 해석"
"전통 공예와 현대 예술의 만남: 한국 전통문화의 새로운 지평"
"K-팝의 진화: 아이돌 그룹의 혁신적인 음악과 패션 변화"
"전통 공예의 현대적 재해석: 한국의 공예가들이 그리는 새로운 미래"
"서울 도심 속 숨겨진 예술의 거리, 재개발 위기 속 창작가들의 외침"
"전통 춤의 현대적 재해석: 젊은 예술가들의 새로운 도전"
"전통 한복, 현대 패션과 만나 재탄생: 한국의 새로운 의상 혁명"
"한국 전통 춤, 현대 패션과의 만남: 새로운 예술적 융합을 탐구하다"
"한국 전통 공예의 현대적 재해석: 젊은 예술가들의 새로운 물결"
"한국

  0%|          | 0/200 [00:00<?, ?it/s]

"월드컵 결승전, 대한민국의 기적을 보다: 한일전의 극적인 순간들"
"세계 최강 축구팀, 한반도에서 우승 도전"
"차세대 축구 스타, 아시아컵 결승 진출의 주역 되다"
"축구 스타의 충격 은퇴 선언, 월드컵 예선 앞두고 팀에 큰 충격파"
"축구 스타, 아시아컵 결승 진출에 빛나는 반전 활약"
"국내 프로축구, 리그 재개 앞두고 선수 부상 현황 점검"
"축구 천재, 월드컵 결승전서 대역전 드라마"
"세계 신기록 수립한 청소년 테니스 선수, 대회서 파란"
"축구 스타, 국제 친선 경기에서 화려한 부활"
"아시아컵 결승전, 한국의 마지막 도전"
"전설의 귀환: 김연아, 올림픽 챔피언십 복귀전서 감격의 금메달"
"월드컵 예선, 한국 축구팀의 파란만장한 여정"
"손흥민, 런던 원정서 환상적인 멀티골로 팀 승리 이끌다"
"월드컵 예선, 한국 축구 팀의 돌풍 예고: 신예들의 반란"
"축구 스타, 첫 번째 국제 토너먼트 우승으로 축구계 새 역사 쓴다"
"축구 스타, 아시안게임서 금빛 중거리 슛!"
"사상 첫 여성 감독, 축구팀의 새로운 역사를 쓰다"
"농구 스타, 올림픽 메달 실패 후 돌연 은퇴 선언"
"프로 축구 리그, 새로운 스타 탄생: 신예 선수의 눈부신 데뷔전"
"축구 천재의 귀환: K리그 복귀로 팬들 excited!"
"손흥민, 챔피언스리그 16강 진출 이끈 결승골의 주인공"
"현지 명문 고교 농구팀, 전국대회 4강 진출"
"세계 최대 축구 대회, 아시아 팀들의 대반격 예고!"
"차세대 축구 스타, 청소년 월드컵서 우승 이끌다"
"축구 스타, 월드컵 결승전에서 눈부신 활약으로 우승 견인"
"축구계 신흥 강자, K리그의 돌풍: 전국 팬들의 기대 속 새로운 도약"
"전설의 귀환: 마이클 조던, NBA 복귀 선언"
"축구 스타, 월드컵 결승전서 극적인 역전골로 우승"
"천재 축구 신동, 국제 청소년 대회서 전설의 기록 경신"
"월드컵 예선, 한국 축구팀의 기적 같은 승리"
"한국 축구, 도쿄 올림픽 금메달로 4회 연속 우승 달성"
"한국 축구, 월드컵 예선서 남미 강적 

...

"인공지능이 바꾸는 미래: 혁신적인 기술의 사회적 영향과 윤리적 딜레마"
"AI 기반 신약 개발, 의료 혁명의 시작"
"AI와 인간의 협업: 미래 업무 환경의 혁신"
"AI 혁명: 미래를 여는 혁신과 도전 과제들"
"미래의 교통 혁명: 자율주행 자동차의 현재와 미래"
"AI 기반 의료 진단 시스템, 의사들의 진료 방식 혁신 예고"
"AI 혁명: 미래의 일자리와 인류의 책임은?"
"AI의 미래: 인간과 로봇의 공존, 어떻게 이루어질까?"
"AI의 진화: 미래 산업을 바꾸는 혁신 기술의 현주소"
"AI 혁명, 일상 속으로: 자율주행차의 미래는?"
"AI로 혁신하는 의료 진단: 정확도와 효율성의 새로운 패러다임"
"AI로 혁신하는 미래: 자율주행차의 새로운 기준"
"AI가 그린 그림, 경매서 5억원 돌파!"
"AI 혁신, 의료 분야의 패러다임 변화 이끌다"
"AI가 인간의 일자리를 대체할까? 미래 직업의 패러다임 변화"
"인공지능의 미래: 일상 속으로 들어온 AI와 그 영향력"
"AI 혁명: 미래의 일자리 풍경은 어떻게 변할까?"
"AI 혁신, 미래 산업의 판도를 바꾸다"
"AI와 인간의 협력: 미래 업무 환경의 혁신적 변화"
"AI로 변신한 도시: 스마트 기술의 미래를 만나다"
"AI 혁신, 의료 분야에서의 무한한 가능성 탐구"
"AI의 진화: 자율주행 자동차가 가져올 미래"
"AI 혁신, 미래 산업의 판도를 바꾸다"
"AI가 바꾸는 미래: 자동화 시대의 일자리와 윤리 문제"
"AI 혁명: 2023년 기술 혁신의 최전선"
"AI 혁명: 미래 직업 지형을 바꾸다"
"인공지능이 바꾸는 미래: 직업 시장의 혁명과 도전"
"AI 시대의 도래: 혁신과 윤리의 경계"
"AI와 인간의 공존: 기술 혁신이 가져올 미래 직업"
"AI가 이끄는 혁신: 자율주행차의 미래와 윤리적 딜레마"
"AI의 혁신: 자율주행차가 바꾸는 미래 교통의 얼굴"
"AI 혁신: 자율주행차가 도시 교통을 바꿀 날이 다가오고 있다"
"AI 혁신, 2023년 산업계를 뒤흔들다"
"AI 혁명: 미래의 직업 풍경

  0%|          | 0/200 [00:00<?, ?it/s]

"중소기업 생존 전략: 위기 속 성장 기회 모색"
"소상공인의 눈물, 임대료 폭탄에 사라진다: 생존을 위한 마지막 저항"
"디지털 화폐 시대, 한국 경제의 새로운 도전과 기회"
"한국 기업, AI 기술 투자 확대로 글로벌 경쟁력 강화"
"한국 기업들, AI 기술 투자 확대: 미래 산업의 핵심"
"대기업 구조조정, 중소기업에 기회 될까?"
"한국, AI 기반 스타트업 열풍... 벤처캐피털 투자 유치 급증"
"경기 침체 속 일자리 창출 대책, 정부의 새로운 전략은?"
"물가 급등에 가계 부담 증가: 생활비 절감 방법은?"
"부동산 시장, 금리 인상에도 불구하고 상승세 지속"
"글로벌 금리 인상, 한국 경제에 미칠 영향은?"
"미중 무역전쟁, 한국 기업들에게 미칠 영향 분석"
"2023년 경제 성장률, 주요 산업 변화와 정부 대응 분석"
"글로벌 금리 인상, 한국 주식시장에 미치는 영향은?"
"글로벌 경제 위기, 한국 산업계의 대응 전략 분석"
"국경 넘어 펼쳐진 디지털 경제: 해외 직접 투자의 새로운 패러다임"
"국내 중소기업, 디지털 전환으로 새로운 성장 동력 모색"
"글로벌 금융 시장, 신흥국 위기 속 안정 찾기 어려운 국면"
"최저임금 인상, 중소기업과 자영업자의 생존 게임"
"디지털 화폐 도입, 한국 경제의 새로운 패러다임"
"한국 반도체 산업, 글로벌 시장 재편기에 새로운 기회 모색"
"기술 혁신이 촉발한 새로운 경제 패러다임: AI와 블록체인의 융합"
"한국 경제, 포스트 팬데믹 시대의 새로운 성장 동력은?"
"한국 GDP, 20년 만에 최고 성장률 기록...미래 산업이 견인"
"중소기업 혁신 지원 정책, 지역 경제 활력 불어넣다"
"한국 경제, 글로벌 인플레이션 속 회복세 조짐"
"미중 무역전쟁 재개, 한국 경제에 미칠 영향은?"
"AI 기술 도입, 한국 경제 혁신의 새로운 바람"
"글로벌 인플레이션, 한국 경제에 미치는 영향 분석"
"신도시 개발로 인해 서울 주택 시장 재편, 중저가 아파트 수요 급증"
"부동산 시장, 금리 인상과 경기 침체 속 새로

  0%|          | 0/200 [00:00<?, ?it/s]

"기후 변화로 인한 국제 분쟁: 북극 빙하의 급격한 감소가 초래하는 새로운 긴장"
"미·중 정상회담: 전략적 대화 속 새로운 무역 협정 발표"
"미-중 정상회담, 새로운 경제 협력 방안 논의"
"G7 정상회의: 기후변화 대응을 위한 글로벌 협력 방안 논의"
"유엔, 파리 기후 협정 탈퇴 선언한 미국의 입장에 공식 반박"
"세계 경제 포럼: 기후 변화와 기술 혁신의 교차점에서"
"G7 정상회의: 글로벌 경제 안정화 방안 논의"
"기후변화로 인한 북극 빙하 감소: 지구 생태계에 미치는 장기적 영향"
"세계 경제 위기, G7 정상회의서 긴급 논의 예정"
"세계 경제 포럼에서의 기후변화 대응 방안 논쟁"
"세계 경제 포럼: 기후 변화 대응과 기술 혁신의 교차로"
"G7 정상회담: 기후변화 대응, 경제 회복 논의"
"세계 정상들, 파리에서 기후변화 긴급 회담 개최"
"G20 정상회의: 기후변화 대응을 위한 공동 선언"
"기후 변화 회의, 각국 지도자들의 긴급 대응책 발표"
"미-중 정상회담서 新무역협정 체결...글로벌 경제 지형 변화 예고"
"세계적인 경제 회복, 주요 국가 간 협력 강화"
"G7 정상회의서 기후변화 대응 합의, 탄소 중립 목표 가속화"
"세계 경제 포럼, 기후 위기 대응 방안 논의"

이 제목은 국제 뉴스 카테고리에 적합하며, 세계적인 행사에서 중요한 글로벌 이슈인 기후 위기가 다뤄지는 내용을 암시합니다. 다양한 국가와 전문가들이 참여하여 해결책을 모색하는 행사라는 점에서 독자의 관심을 끌 수 있을 것입니다.
"세계 정상회담서 기후변화 대응 새로운 합의 도출"
"세계 경제, 러-우 전쟁 장기화로 심각한 불확실성 직면"
"미-중 무역전쟁: 새로운 관세 부과로 글로벌 경제 긴장 고조"
"세계 정상들, 파리 기후협정 재합의 회의 개최"
"세계 정상회의서 기후변화 대응 방안 발표: 각국의 협력과 도전"
"G7 정상회의, 기후변화 대응 공동 성명 발표"
"세계 경제 포럼: 디지털 혁신이 가져올 미래"
"G7 정상회담, 글로벌 경제 회복 방안 논의"
"기후 변화

In [11]:
gen_df = pd.DataFrame(generated_article).sample(frac=1).reset_index(drop=True)
gen_df.to_csv("../data/gen_article_exaone.csv", index=False, encoding="utf-8-sig")

In [12]:
print(len(gen_df))

1400


In [ ]:
def relabel(text: str, subject: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 신문 기사가 한 개가 들어오고 후보 주제둘이 주어집니다. 각 주제들은 개행 문자로 구분이 됩니다. 입력으로 들어온 후보 주제들 중에서 가장 신문 기사에 어울리는 주제 단 한 개를 출력하세요. 설명을 붙이지 말고 주제 하나만을 출력하세요. 복수의 주제를 출력하면 안됩니다."},
    {"role": "user", "content": text},
    {"role": "user", "content": subject}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=4096,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    if '\n' in result:
        result = result.split("\n")[0]
    del inputs, outputs, generated_text
        
    return result

In [ ]:
final_topic = ""
for subtopic in topics:
    final_topic += subtopic + "\n"

with open("../data/original_nanoised.csv", 'r') as f:
    df2 = pd.read_csv(f)

label_filtered = []
for i, row in tqdm(df2.iterrows(), desc="Rerabelling", total=len(df2)):
    result = relabel(row['text'], final_topic)
    idx = -1
    for k, subtopic in enumerate(topics):
        if subtopic == result:
            idx = k
            break
    label_filtered.append(
        {
            'ID': row['ID'],
            'text': row['text'],
            'target': idx
        }
    )
    

In [ ]:
df3 = pd.DataFrame(label_filtered)
df3.to_csv("../data/nanoised_label_filtered.csv", index=False, encoding="utf-8-sig")

In [ ]:
print(len(df3))